In [7]:
from pymongo import MongoClient
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
from collections import Counter
import pandas as pd
import numpy as np


In [8]:
#connecting to MongoClient through Studio3T
client = MongoClient(port = 27017)
db = client["Weather_data"]
weather_data = []
#grabbing all datasets (json files) via Studio3T
for n in range(0,7):
    weather_data.append(db[f'{n}'])

In [9]:
#Shows how one element of the weather_data
print(type(weather_data[1]))
print(weather_data[1])

<class 'pymongo.collection.Collection'>
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Weather_data'), '1')


In [10]:
for i in weather_data:
    for t in i.find({}).limit(1):
        print(t)
    print("\n")


{'_id': ObjectId('62f59b5c84db5e328ee97c23'), 'LocalObservationDateTime': '2022-08-04T09:55:00+08:00', 'EpochTime': 1659578100, 'WeatherText': ' MOstlY ClOudy ', 'WeatherIcon': 6, 'HasPrecipitation': False, 'PrecipitationType': None, 'IsDayTime': True, 'Temperature': {'Metric': {'Value': 12.2, 'Unit': 'C', 'UnitType': 17}, 'Imperial': {'Value': 54.0, 'Unit': 'F', 'UnitType': 18}}, 'city': ' pERTH '}


{'_id': ObjectId('62f59b5c84db5e328ee97c24'), 'LocalObservationDateTime': '2022-08-04T11:25:00+09:30', 'EpochTime': 1659578100, 'WeatherText': ' MOStlY CloUdy ', 'WeatherIcon': 6, 'HasPrecipitation': False, 'PrecipitationType': None, 'IsDayTime': True, 'Temperature': {'Metric': {'Value': 26.1, 'Unit': 'C', 'UnitType': 17}, 'Imperial': {'Value': 79.0, 'Unit': 'F', 'UnitType': 18}}, 'city': ' darwIN '}


{'_id': ObjectId('62f59b5c84db5e328ee97c83'), 'LocalObservationDateTime': '2022-08-04T11:55:00+10:00', 'EpochTime': 1659578100, 'WeatherText': ' rAIN ', 'WeatherIcon': 18, 'HasPrecipitation

### Findings
* Weather and city keys have mixed upper and lower cased values eg PeRtH and CloUDY
* In each document Data includes both Metric (Celcius) and Imperial (Fahrenheit) for weather data

Must update the text for City and WeatherText, with do this in Part a and b.

# Part a - Average temperature per city

## Part a - With Metric

In [11]:
#Writing a new text file that searches through each document (json files) and grabs the city (lowercase) and the metric temperature value (seperated by colon)

with open('part_a_metric.txt', 'w', encoding='utf-8') as f:
    for i in weather_data:
        for t in i.find({}):
            f.write(t["city"].lower())
            f.write(':')
            #convert to string as write does not pass integers or floats
            f.write(str(t["Temperature"]["Metric"]["Value"]))
            f.write("\n")


## Part a - With Imperial

In [12]:
with open('part_a_imperial.txt', 'w', encoding='utf-8') as f:
    for i in weather_data:
        for t in i.find({}):
            f.write(t["city"].lower())
            f.write(':')
            #convert to string as write does not pass integers or floats
            f.write(str(t["Temperature"]["Imperial"]["Value"]))
            f.write("\n")

# Part b - Top three most common 'weather text' for each city

In [13]:
#similar to part a however we will grab city and weathertext 
with open('part_b_data.txt', 'w', encoding='utf-8') as f:
    for i in weather_data:
        for t in i.find({}):
            f.write(t["city"].lower())
            f.write(':')
            #weathertext - use strip() to remove the spacing from front and back of the string, add ')' for future purposes
            f.write(t["WeatherText"].lower().strip()+')')
            f.write("\n")

In [14]:
#converting my text into a csv (pandas) seperated by the colon and no header. I then add columns to the pandas data frame
data = pd.read_csv('part_b_data.txt', sep=":", header=None)
data.columns = ['City', 'WeatherText']
data.head()

,City,WeatherText
0,perth,mostly cloudy)
1,perth,cloudy)
2,perth,light rain shower)
3,perth,light rain shower)
4,perth,rain shower)


In [15]:
#We replace spacings with underscores and remove the ')' from what we had previously
data['WeatherText'] = data['WeatherText'].str.replace(" ", "_").str.replace(')', ' ')
data.head()

C:\Users\Chris\AppData\Local\Temp\ipykernel_32392\1434988618.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['WeatherText'] = data['WeatherText'].str.replace(" ", "_").str.replace(')', ' ')


,City,WeatherText
0,perth,mostly_cloudy
1,perth,cloudy
2,perth,light_rain_shower
3,perth,light_rain_shower
4,perth,rain_shower


In [16]:
#grabbing all cities and storing within a sorted list
cities = sorted(data['City'].unique().tolist())
cities

[' brisbane ',
 ' canberra ',
 ' darwin ',
 ' hobart ',
 ' melbourne ',
 ' perth ',
 ' sydney ']

In [17]:
#Group Citys by Sum to grab all Weather text for each city.
city_weather_dat = data.groupby("City").sum()
city_weather_dat

,WeatherText
City,
brisbane,partly_sunny partly_sunny partly_sunny partly_...
canberra,rain rain heavy_rain heavy_rain thunderstorm c...
darwin,mostly_cloudy partly_sunny sunny sunny sunny c...
hobart,clouds_and_sun partly_sunny mostly_sunny mostl...
melbourne,mostly_cloudy mostly_cloudy mostly_cloudy most...
perth,mostly_cloudy cloudy light_rain_shower light_r...
sydney,cloudy cloudy cloudy mostly_cloudy cloudy clou...


In [18]:
#converting the weather text data into arrays (each city's weather text is an array)
list_weather = city_weather_dat.to_numpy()
tokenized_weather = []
#Word tokenizing the weather text to develop a list of each word
for i in list_weather:
    for k in i:
        tokenized_weather.append(word_tokenize(k))
tokenized_weather[1][:5]


['rain', 'rain', 'heavy_rain', 'heavy_rain', 'thunderstorm']

In [19]:
#I iterate through the tokenized_weather list and collect each cities top 3 most common as add them to a new list
three_most_common = []
for i in tokenized_weather:
    three_most_common.append(Counter(i).most_common(3))
three_most_common[0]

[('partly_sunny', 16), ('foggy', 10), ('ground_fog', 10)]

In [20]:
#I combined the city names most common weather text together through a dictionary
results = {}
for i in range(len(cities)):
    results.update({cities[i] : three_most_common[i]})
print(results)

{' brisbane ': [('partly_sunny', 16), ('foggy', 10), ('ground_fog', 10)], ' canberra ': [('mostly_cloudy', 8), ('thunderstorm', 6), ('cloudy', 6)], ' darwin ': [('partly_sunny', 10), ('clear', 8), ('cloudy', 8)], ' hobart ': [('cloudy', 12), ('mostly_clear', 8), ('mostly_sunny', 6)], ' melbourne ': [('mostly_cloudy', 48)], ' perth ': [('cloudy', 10), ('light_rain_shower', 10), ('rain_shower', 8)], ' sydney ': [('clear', 18), ('cloudy', 14), ('sunny', 8)]}


In [21]:
# Now I remove the underscore so we get a space inbetween words again
for k,v in results.items():
    for w, n in v:
        w= w.replace("_", " ")

In [22]:
#Converting the dictionary to a csv
part_b_final = pd.DataFrame.from_dict(results, orient='index')
print(part_b_final)
part_b_final.to_csv('output.part_b.csv', index=True)

                               0                        1                  2
 brisbane     (partly_sunny, 16)              (foggy, 10)   (ground_fog, 10)
 canberra     (mostly_cloudy, 8)        (thunderstorm, 6)        (cloudy, 6)
 darwin       (partly_sunny, 10)               (clear, 8)        (cloudy, 8)
 hobart             (cloudy, 12)        (mostly_clear, 8)  (mostly_sunny, 6)
 melbourne   (mostly_cloudy, 48)                     None               None
 perth              (cloudy, 10)  (light_rain_shower, 10)   (rain_shower, 8)
 sydney              (clear, 18)             (cloudy, 14)         (sunny, 8)
